<a href="https://colab.research.google.com/github/brinrs/SportsGamePredictorMVP/blob/main/Sports_Game_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# **Import the libraries**

In [ ]:
!apt-get update
!pip install nba_api
!pip install selenium
!pip install requests beautifulsoup4 pandas
!pip install webdriver-manager

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
import os
import pandas as pd
import requests
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import leaguedashteamstats, commonteamroster
from nba_api.stats.endpoints import playergamelogs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# **Load the datasets**

## **Match up stats**

In [ ]:
from nba_api.stats.endpoints import LeagueGameFinder

seasons = [str(year) + '-' + str(year+1)[2:] for year in range(2015, 2024)]
all_games = []
for season in seasons:
    gamefinder = LeagueGameFinder(season_nullable=season)
    games = gamefinder.get_data_frames()[0]
    games['SEASON'] = season
    all_games.append(games)

df = pd.concat(all_games)
season_col = df.pop("SEASON")  # Remove SEASON column
df.insert(0, "SEASON", season_col)
df.head()

,SEASON,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,2015-16,42015,1610612744,GSW,Golden State Warriors,0041500407,2016-06-19,GSW vs. CLE,L,239,...,0.769,7,32,39,22,7,5,10,23,-4.0
1,2015-16,42015,1610612739,CLE,Cleveland Cavaliers,0041500407,2016-06-19,CLE @ GSW,W,241,...,0.840,9,39,48,17,7,6,11,15,4.0
2,2015-16,42015,1610612744,GSW,Golden State Warriors,0041500406,2016-06-16,GSW @ CLE,L,238,...,0.690,9,26,35,19,5,3,14,25,-14.0
3,2015-16,42015,1610612739,CLE,Cleveland Cavaliers,0041500406,2016-06-16,CLE vs. GSW,W,240,...,0.781,8,37,45,24,12,7,10,25,14.0
4,2015-16,42015,1610612739,CLE,Cleveland Cavaliers,0041500405,2016-06-13,CLE @ GSW,W,241,...,0.609,8,33,41,15,11,9,16,22,15.0


**Data cleaning**

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

missing_values

Missing Values:
 WL         5
FG_PCT     2
FG3_PCT    3
FT_PCT     9
dtype: int64


,0
SEASON,0
SEASON_ID,0
TEAM_ID,0
TEAM_ABBREVIATION,0
TEAM_NAME,0
GAME_ID,0
GAME_DATE,0
MATCHUP,0
WL,5
MIN,0


In [ ]:
df.loc[:, 'FG_PCT'] = df['FG_PCT'].fillna(df['FG_PCT'].median())
df.loc[:, 'FG3_PCT'] = df['FG3_PCT'].fillna(df['FG3_PCT'].median())
df.loc[:, 'FT_PCT'] = df['FT_PCT'].fillna(df['FT_PCT'].median())

# Fill missing categorical values with the most frequent value
df.loc[:, 'WL'] = df['WL'].fillna(df['WL'].mode()[0])

# Verify missing values are fixed
missing_values = df.isnull().sum()
print("Missing Values After Fix:\n", missing_values[missing_values > 0])

Missing Values After Fix:
 Series([], dtype: int64)


## **Team stats**

In [ ]:
seasons = [str(year) + '-' + str(year+1)[2:] for year in range(2015, 2024)]

All_teams = []
for season in seasons:
    team_stats = leaguedashteamstats.LeagueDashTeamStats(season=season)
    df_teams = team_stats.get_data_frames()[0]
    df_teams['SEASON'] = season
    All_teams.append(df_teams)

df_teams = pd.concat(All_teams)
season_col = df_teams.pop("SEASON")  # Remove SEASON column
df_teams.insert(0, "SEASON", season_col)
df_teams.head()

,SEASON,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,2015-16,1610612737,Atlanta Hawks,82,48,34,0.585,3966.0,3168,6923,...,24,2,22,3,5,14,7,29,12,7
1,2015-16,1610612738,Boston Celtics,82,48,34,0.585,3956.0,3216,7318,...,6,6,12,2,22,24,29,10,5,8
2,2015-16,1610612751,Brooklyn Nets,82,21,61,0.256,3951.0,3136,6920,...,23,15,19,19,25,18,2,28,26,28
3,2015-16,1610612766,Charlotte Hornets,82,48,34,0.585,3971.0,3036,6922,...,14,19,1,22,12,21,3,14,11,9
4,2015-16,1610612741,Chicago Bulls,82,42,40,0.512,3981.0,3165,7170,...,3,12,13,29,8,27,5,26,21,18


In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 Series([], dtype: int64)


## **Player stats**

In [ ]:
seasons = [str(year) + '-' + str(year+1)[2:] for year in range(2015, 2024)]

All_players = []
for season in seasons:
    player_stats = playergamelogs.PlayerGameLogs(season_nullable=season)
    df_players = player_stats.get_data_frames()[0]
    df_players['SEASON'] = season
    All_players.append(df_players)

df_players = pd.concat(All_players)
season_col = df_players.pop("SEASON")  # Remove SEASON column
df_players.insert(0, "SEASON", season_col)
df_players.head()

,SEASON,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,MIN_SEC
0,2015-16,2015-16,201975,Jodie Meeks,Jodie,1610612765,DET,Detroit Pistons,0021501220,2016-04-13T00:00:00,...,1,7786,2594,4633,6818,2107,76,4936,1.0,26:32
1,2015-16,2015-16,203110,Draymond Green,Draymond,1610612744,GSW,Golden State Warriors,0021501227,2016-04-13T00:00:00,...,4972,12373,9124,1041,4423,2107,76,3722,1.0,29:47
2,2015-16,2015-16,202703,Nikola Mirotic,Nikola,1610612741,CHI,Chicago Bulls,0021501222,2016-04-13T00:00:00,...,22104,2748,275,4633,61,2107,76,56,1.0,39:00
3,2015-16,2015-16,203085,Austin Rivers,Austin,1610612746,LAC,LA Clippers,0021501229,2016-04-13T00:00:00,...,11390,7786,4835,20888,4063,2107,76,4936,1.0,30:57
4,2015-16,2015-16,1717,Dirk Nowitzki,Dirk,1610612742,DAL,Dallas Mavericks,0021501223,2016-04-13T00:00:00,...,11390,1539,3065,15372,5452,2107,76,4936,1.0,33:48


In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 Series([], dtype: int64)


## **Elo Ratings & Advanced Metrics**

In [ ]:
elo_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv"

# Read the dataset
df_elo = pd.read_csv(elo_url)
df_elo = df_elo[df_elo['year_id'] >= 2015]
df_elo.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
123692,61847,201410280LAL,NBA,1,2015,10/28/2014,1,0,HOU,Rockets,...,51.586834,LAL,Lakers,90,1421.9985,1410.6588,A,W,0.605553,NaN
123693,61847,201410280LAL,NBA,0,2015,10/28/2014,1,0,LAL,Lakers,...,31.337477,HOU,Rockets,108,1596.4641,1607.8038,H,L,0.394447,NaN
123694,61848,201410280NOP,NBA,0,2015,10/28/2014,1,0,NOP,Pelicans,...,36.695789,ORL,Magic,84,1359.4265,1353.2910,H,W,0.757419,NaN
123695,61848,201410280NOP,NBA,1,2015,10/28/2014,1,0,ORL,Magic,...,25.881901,NOP,Pelicans,101,1457.2185,1463.3540,A,L,0.242581,NaN
123696,61849,201410280SAS,NBA,0,2015,10/28/2014,1,0,SAS,Spurs,...,60.040932,DAL,Mavericks,100,1592.0101,1590.3984,H,W,0.767531,NaN


## **Game Context Data**

Current Season Injury Info

In [ ]:
# ✅ Remove old ChromeDriver versions
!apt-get remove -y chromium-browser chromium-chromedriver
!rm -rf /usr/bin/chromedriver

# ✅ Install latest Chrome for Colab
!wget -q -O /tmp/chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y /tmp/chrome.deb

# ✅ Get the latest matching ChromeDriver version
chrome_version = !google-chrome --version
chrome_version = chrome_version[0].split()[2]  # Extract version number
chromedriver_url = f"https://storage.googleapis.com/chrome-for-testing-public/{chrome_version}/linux64/chromedriver-linux64.zip"

# ✅ Download & Install ChromeDriver
!wget -q -O /tmp/chromedriver.zip {chromedriver_url}
!unzip -o /tmp/chromedriver.zip -d /usr/bin/
!chmod +x /usr/bin/chromedriver

# ✅ Verify installation
!google-chrome --version
!chromedriver --version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'chromium-browser' is not installed, so not removed
Package 'chromium-chromedriver' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  apparmor libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of '/tmp/chrome.deb'
google-chrome-stable is already the newest version (133.0.6943.141-1).
The following packages were automatically installed and are no longer required:
  apparmor libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
Archive:  /tmp/chromedriver.zi

In [ ]:
# ✅ Remove any previous installations
!rm -rf /usr/bin/chromedriver

# ✅ Move ChromeDriver to the correct location
!mv /usr/bin/chromedriver-linux64/chromedriver /usr/bin/chromedriver

# ✅ Grant execution permissions
!chmod +x /usr/bin/chromedriver

# ✅ Verify installation
!chromedriver --version

ChromeDriver 133.0.6943.141 (2a5d6da0d6165d7b107502095a937fe7704fcef6-refs/branch-heads/6943@{#1912})


In [ ]:
# ✅ Set Chrome options for Colab
options = webdriver.ChromeOptions()
options.binary_location = "/usr/bin/google-chrome"  # Correct path for Chrome
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# ✅ Use the correct ChromeDriver path
service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# ✅ Open ESPN NBA Injury Page
url = "https://www.espn.com/nba/injuries"
driver.get(url)

# ✅ Wait for JavaScript to load
time.sleep(5)
teams = driver.find_elements(By.XPATH, "//div[contains(@class, 'ResponsiveTable')]")

injuries = []

# ✅ Loop through EACH team section
for team in teams:
    try:
        # ✅ Extract team name
        team_name = team.find_element(By.XPATH, ".//span[contains(@class, 'teamName')]").text.strip()
        print(f"Extracting injuries for {team_name}...")  # Debugging

        # ✅ Extract player rows **inside this team section**
        players = team.find_elements(By.XPATH, ".//tbody/tr")

        for player in players:
            cols = player.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 5:  # Ensure we have enough columns
                # ✅ Extract player name & profile link
                try:
                    player_elem = cols[0].find_element(By.TAG_NAME, "a")
                    player_name = player_elem.text.strip()
                    player_link = player_elem.get_attribute("href")
                except:
                    player_name = cols[0].text.strip()
                    player_link = None

                # ✅ Extract position
                position = cols[1].text.strip()

                # ✅ Extract estimated return date
                return_date = cols[2].text.strip()

                # ✅ Extract injury status (e.g., "Day-to-Day", "Out")
                try:
                    status = cols[3].find_element(By.TAG_NAME, "span").text.strip()
                except:
                    status = cols[3].text.strip()

                # ✅ Extract injury description/comment
                comment = cols[4].text.strip() if len(cols) > 4 else ""

                # ✅ Store extracted data
                injuries.append({
                    "Team": team_name,
                    "Player": player_name,
                    "Position": position,
                    "Return Date": return_date,
                    "Status": status,
                    "Comment": comment,
                    "Player Profile": player_link
                })

    except Exception as e:
        print(f"Error processing {team_name}: {e}")

# ✅ Convert to DataFrame
df_injuries = pd.DataFrame(injuries)

Extracting injuries for Atlanta Hawks...
Extracting injuries for Boston Celtics...
Extracting injuries for Brooklyn Nets...
Extracting injuries for Chicago Bulls...
Extracting injuries for Cleveland Cavaliers...
Extracting injuries for Dallas Mavericks...
Extracting injuries for Denver Nuggets...
Extracting injuries for Detroit Pistons...
Extracting injuries for Golden State Warriors...
Extracting injuries for Houston Rockets...
Extracting injuries for Indiana Pacers...
Extracting injuries for LA Clippers...
Extracting injuries for Los Angeles Lakers...
Extracting injuries for Memphis Grizzlies...
Extracting injuries for Miami Heat...
Extracting injuries for Milwaukee Bucks...
Extracting injuries for Minnesota Timberwolves...
Extracting injuries for New Orleans Pelicans...
Extracting injuries for New York Knicks...
Extracting injuries for Oklahoma City Thunder...
Extracting injuries for Orlando Magic...
Extracting injuries for Philadelphia 76ers...
Extracting injuries for Phoenix Suns.

In [ ]:
df_injuries

,Team,Player,Position,Return Date,Status,Comment,Player Profile
0,Atlanta Hawks,Trae Young,PG,Feb 26,Day-To-Day,Feb 25: Young (Achilles) has been listed as pr...,https://www.espn.com/nba/player/_/id/4277905/t...
1,Atlanta Hawks,Larry Nance Jr.,PF,Apr 1,Out,Feb 12: Nance (knee) will have a non-displaced...,https://www.espn.com/nba/player/_/id/2580365/l...
2,Atlanta Hawks,Vit Krejci,G,Mar 14,Out,Feb 12: The Hawks announced Wednesday that Kre...,https://www.espn.com/nba/player/_/id/4578893/v...
3,Atlanta Hawks,Jalen Johnson,SF,Oct 1,Out,Jan 29: Johnson has suffered a torn labrum in ...,https://www.espn.com/nba/player/_/id/4701230/j...
4,Atlanta Hawks,Kobe Bufkin,G,Oct 1,Out,Dec 18: Bufkin will miss the remainder of the ...,https://www.espn.com/nba/player/_/id/4683736/k...
...,...,...,...,...,...,...,...
92,Utah Jazz,Collin Sexton,PG,Feb 28,Out,Feb 25: Sexton (ankle) has been ruled out for ...,https://www.espn.com/nba/player/_/id/4277811/c...
93,Utah Jazz,Taylor Hendricks,F,Oct 1,Out,Nov 6: Hendricks underwent surgery on his brok...,https://www.espn.com/nba/player/_/id/4684806/t...
94,Washington Wizards,Alex Sarr,PF,Feb 26,Day-To-Day,Feb 25: Sarr (ankle) is listed as questionable...,https://www.espn.com/nba/player/_/id/5160992/a...
95,Washington Wizards,Malcolm Brogdon,PG,Mar 1,Out,Feb 25: Brogdon (ankle) has been ruled out for...,https://www.espn.com/nba/player/_/id/2566769/m...


## **Historical Injury and Off Info**

**Betting Market Data**

In [ ]:
import requests

# Fetch NBA betting odds from The Odds API (replace with actual API key)
url = "https://api.the-odds-api.com/v4/sports/basketball_nba/odds?regions=us&apiKey=YOUR_API_KEY"
response = requests.get(url)

# Convert to DataFrame
df_odds = pd.DataFrame(response.json())

In [ ]:
class Point:
    def __init__(self, x, y):
      self.x = x
      self.y = y

def main():
    p = Point(x=2024, y=10) # you can assume this works
    not_p = p # not_p is a reference to p
    not_p.x = 2015 # modifying not_p also modifies p
    p.x += 2 # p.x is now 2017

    m = p.y # m is assigned p.y (which is 10)
    m += 1 # modifying m does not affect p.y

    print(p.x) # prints 2017
    print(m) # prints 11
    print(p.y) # prints 10

main()